In [ ]:
%matplotlib inline

# Execution statistics as a Gantt chart.

When
the global attribute [ExecutionStatistics.is_time_stamps_enabled][gemseo.core.execution_statistics.ExecutionStatistics.is_time_stamps_enabled] is `True`
(default: `False`),
the global attribute [ExecutionStatistics.time_stamps][gemseo.core.execution_statistics.ExecutionStatistics.time_stamps] is
a dictionary of the form `{name: (initial_time, final_time, is_linearization)}`
to store
the initial and final times of each execution and linearization of each discipline.

The [create_gantt_chart()][gemseo.post.core.gantt_chart.create_gantt_chart] function can display this dictionary
in the form of a [Gantt chart](https://en.wikipedia.org/wiki/Gantt_chart).

In this example,
we illustrate the use of this function
on the Sobieski's SSBJ problem.


In [ ]:
from __future__ import annotations

from gemseo import configure
from gemseo import create_discipline
from gemseo import create_scenario
from gemseo.core.execution_statistics import ExecutionStatistics
from gemseo.post.core.gantt_chart import create_gantt_chart
from gemseo.problems.mdo.sobieski.core.design_space import SobieskiDesignSpace

## Enable the recording of statistics




In [ ]:
configure(enable_discipline_statistics=True)

## Create the scenario

First, we define the Sobieski's SSBJ problem as a scenario.

For this,
we instantiate the disciplines:



In [ ]:
disciplines = create_discipline([
    "SobieskiPropulsion",
    "SobieskiAerodynamics",
    "SobieskiStructure",
    "SobieskiMission",
])

as well as the design space:



In [ ]:
design_space = SobieskiDesignSpace()

Then,
given these disciplines and design space,
we build an MDO scenario using the MDF formulation
in order to maximize the range `"y_4"` with respect to the design variables:



In [ ]:
scenario = create_scenario(
    disciplines,
    "y_4",
    design_space,
    formulation_name="MDF",
    maximize_objective=True,
)

and satisfy the inequality constraints
associated with the outputs `"g_1"`, `"g_2"` and `"g_3"`:



In [ ]:
for constraint in ["g_1", "g_2", "g_3"]:
    scenario.add_constraint(constraint, constraint_type="ineq")

## Execute the scenario

By default,
a scenario does *not* produce execution statistics.
We need to enable this *global* mechanism before executing the scenario:



In [ ]:
ExecutionStatistics.is_time_stamps_enabled = True

!!! warning

    This mechanism is *global*
    and shall be modified from the [ExecutionStatistics][gemseo.core.execution_statistics.ExecutionStatistics] class
    (not from an [ExecutionStatistics][gemseo.core.execution_statistics.ExecutionStatistics] instance).

The scenario can now be executed
using the SLSQP optimization algorithm and a maximum of 10 iterations:



In [ ]:
scenario.execute(algo_name="SLSQP", max_iter=10)

!!! info "See also"

    The formulation settings passed to [create_scenario()][gemseo.create_scenario]
    and the algorithm settings passed to [BaseScenario.execute()][gemseo.scenarios.base_scenario.BaseScenario.execute]
    can be provided via Pydantic models.
    For more information,
    see [this page][algorithm-settings] and [this page][formulation-settings].

## Plot the Gantt chart

Lastly,
we plot the Gantt chart from the global [ExecutionStatistics.time_stamps][gemseo.core.execution_statistics.ExecutionStatistics.time_stamps]:



In [ ]:
create_gantt_chart(save=False, show=True)

This graph shows the evolution over time:

- the execution and linearizations of the different user disciplines,
  *e.g.* [SobieskiAerodynamics][gemseo.problems.mdo.sobieski.disciplines.SobieskiAerodynamics],
- the execution and linearizations of the different process disciplines,
  *e.g.* [MDAJacobi][gemseo.mda.jacobi.MDAJacobi],
- the execution of the scenario.



## Disable recording

Finally,
we disable the recording of time stamps for other executions:



In [ ]:
ExecutionStatistics.is_time_stamps_enabled = False

!!! note

    As this reset [ExecutionStatistics.time_stamps][gemseo.core.execution_statistics.ExecutionStatistics.time_stamps] to `None`,
    the [create_gantt_chart()][gemseo.post.core.gantt_chart.create_gantt_chart] function can no longer be used.
    Set [ExecutionStatistics.is_time_stamps_enabled][gemseo.core.execution_statistics.ExecutionStatistics.is_time_stamps_enabled] to `True`
    and execute or linearize some disciplines
    so that you can use it again.

